<a href="https://colab.research.google.com/github/Pravalika320/Pravalika/blob/main/midterm/midterm-2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, resp_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, int, int, int, float, tr, str, str) -> float
    The PRIEST function will calculate the 30-day probability of an adverse outcome. It uses definition provided by evidence from MDCalc website https://www.mdcalc.com/calc/10369/priest-covid-19-clinical-severity-score#evidence
    Returns the probability of an adverse outcome, between 0 and 1 for valid inputs such as 'female', 'male', 'Female', 'Male',
    'FEMALE', 'MALE', 'MalE' , 'FemalE'. And returns nothing if the input is invalid.


    >>> priest('Male', 18, 20, 0.94, 105.0, 115.0, 35.4, 'alert', 'air', 'unrestricted normal activity')
    0.03

    >>> priest('male', 85, 10.0, 0.96, 90.0, 101.0, 38.1, 'confused or not alert', 'supplemental oxygen', 'unrestricted normal activity')
    0.46

    >>> priest('Invalid', 90, 18, 0.96, 90, 100, 37.5, 'alert', 'air', 'limited self-care')

    """

    sex = sex.lower()
    if sex not in ['male', 'female']:
      return None
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()


    sex_points = 1 if sex == 'male' else 0

    age_points = 0
    if 16 <= age <= 49:
        age_points = 0
    elif 50 <= age <= 65:
        age_points = 2
    elif 66 <= age <= 80:
        age_points = 3
    elif age > 80:
        age_points = 4

    resp_rate_points = 3 if resp_rate < 9 else 1 if 9 <= resp_rate <= 11 else 0 if 12 <= resp_rate <= 20 else 2

    oxygen_points = 0 if oxygen_saturation > 0.95 else 1 if 0.94 <= oxygen_saturation <= 0.95 else 2

    heart_rate_points = 3 if heart_rate < 41 else 1 if 41 <= heart_rate <= 50 else 0 if 51 <= heart_rate <= 90 else 1 if 91 <= heart_rate <= 110 else 2 if 111 <= heart_rate <= 130 else 3

    systolic_bp_points = 3 if systolic_bp < 91 else 2 if 91 <= systolic_bp <= 100 else 1 if 101 <= systolic_bp <= 110 else 0

    temp_points = 3 if temperature < 35.1 else 1 if 35.1 <= temperature <= 36.0 else 0 if 36.1 <= temperature <= 38.0 else 1 if 38.1 <= temperature <= 39.0 else 2

    alertness_points = 0 if alertness == 'alert' else 3

    inspired_oxygen_points = 0 if inspired_oxygen == 'air' else 2

    performance_points = {
        'unrestricted normal activity': 0,
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
        'bed/chair bound, no self-care': 4
    }.get(performance_status, None)

    if performance_points is None:
        return None

    # Calculating the score
    priest_score = (
        sex_points + age_points + resp_rate_points + oxygen_points +
        heart_rate_points + systolic_bp_points + temp_points +
        alertness_points + inspired_oxygen_points + performance_points
    )

    # Calculating the 30-day probability
    probability_map = {
        0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22,
        9: 0.26, 10: 0.29, 11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55,
        17: 0.59, 18: 0.59, 19: 0.59, 20: 0.59, 21: 0.59, 22: 0.59, 23: 0.59, 24: 0.59, 25: 0.59,
    }

    return probability_map.get(priest_score, 0.99)


In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('Male', 18, 20, 0.94, 105.0, 115.0, 35.4, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.03
ok
Trying:
    priest('male', 85, 10.0, 0.96, 90.0, 101.0, 38.1, 'confused or not alert', 'supplemental oxygen', 'unrestricted normal activity')
Expecting:
    0.46
ok
Trying:
    priest('Invalid', 90, 18, 0.96, 90, 100, 37.5, 'alert', 'air', 'limited self-care')
Expecting nothing
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests

def find_hospital(age, sex, risk):
    """ (int, str, float) -> str
    To find the hospital based on factors like age, sex, and risk, the function find_hospital will make a request to a REST web service.
    If the call is successful, it returns the name of the hospital; if not, it returns None.

    URL:
    https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1

    Test cases:
    >>> find_hospital(40, 'male', 0.18)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(40, 'female', 0.03)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(65, 'male', 0.18)
    'Wesley Woods Geriatric Hospital'
    """

    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    params = {
        'age': age,
        'sex': sex.lower(),
        'risk_pct': risk
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        hospital_data = response.json()

        return hospital_data.get('hospital', None)

    except requests.exceptions.RequestException:
        return None

    except Exception:
        return None

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.18)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40, 'female', 0.03)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(65, 'male', 0.18)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [7]:
import requests
def get_address(hospital_name):
    """ (str) -> str
    This function retrieves the address of a hospital based on its name from the JSON file https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-, returning None if not found.


    >>> get_address("WELLSTAR ATLANTA MEDICAL CENTER")
    '303 PARKWAY DRIVE NE'

    >>> get_address("LIGHTHOUSE CARE CENTER OF AUGUSTA")
    '3100 PERIMETER PARKWAY'

    >>> get_address("APPLING HEALTHCARE SYSTEM")
    '163 EAST TOLLISON STREET'

    """

    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    try:
        response = requests.get(json_url)
        response.raise_for_status()
        data = json.loads(response.text)

        if hospital_name in data:
            hospital_info = data[hospital_name]
            address = hospital_info.get("ADDRESS")

            if address:
                return address

        # Return None if the hospital name is not found or if the address is not available
        return None

    except requests.exceptions.RequestException as e:
        # Handle request-related errors and return None
        return None

    except Exception as e:
        # Handle other unexpected errors and return None
        return None

In [8]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("WELLSTAR ATLANTA MEDICAL CENTER")
Expecting:
    '303 PARKWAY DRIVE NE'
**********************************************************************
File "__main__", line 7, in NoName
Failed example:
    get_address("WELLSTAR ATLANTA MEDICAL CENTER")
Expected:
    '303 PARKWAY DRIVE NE'
Got nothing
Trying:
    get_address("LIGHTHOUSE CARE CENTER OF AUGUSTA")
Expecting:
    '3100 PERIMETER PARKWAY'
**********************************************************************
File "__main__", line 10, in NoName
Failed example:
    get_address("LIGHTHOUSE CARE CENTER OF AUGUSTA")
Expected:
    '3100 PERIMETER PARKWAY'
Got nothing
Trying:
    get_address("APPLING HEALTHCARE SYSTEM")
Expecting:
    '163 EAST TOLLISON STREET'
**********************************************************************
File "__main__", line 13, in NoName
Failed example:
    get_address("APPLING HEALTHCARE SYSTEM")
Expected:
    '163 EAST TOLLISON STREET'
Got nothing


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [9]:
import requests
import csv

def process_people(file_location):
    """
    (str) -> dict
    Processes patient data from the file in given location and retrieves risk, hospital and address information.
    """
    data = {}

    try:
        response = requests.get(file_location)
        response.raise_for_status()

        reader = csv.DictReader(response.text.splitlines(), delimiter='|')

        for row in reader:
            patient_number = row['patient']
            sex = row['sex']
            age = int(row['age'])
            breath = float(row['breath'])
            o2sat = float(row['o2 sat'])
            heart = float(row['heart'])
            systolic = float(row['systolic bp'])
            temp = float(row['temp'])
            alertness = row['alertness']
            inspired = row['inspired']
            status = row['status']

            risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

            hospital = find_hospital(age, sex, risk)

            hospital_upper = hospital.upper()

            address = get_address(hospital_upper)

            patient_data = [
                sex,
                age,
                breath,
                o2sat,
                heart,
                systolic,
                temp,
                alertness,
                inspired,
                status,
                risk,
                hospital,
                address,
            ]

            data[patient_number] = patient_data

    except Exception as e:
        print("An error occurred while calling the existing functions:", e)

    return data

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [10]:
import requests
import json
def retrieve_processed_data(file_url):
    response = requests.get(file_url)
    if response.status_code == 200:
        return json.loads(response.text)
    return None

processed_data = retrieve_processed_data("https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5")

# Fetch the results from given JSON file URL:
given_json_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
given_json_results = retrieve_processed_data(given_json_url)

if processed_data is not None and given_json_results is not None:

    for patient_number, expected_data in given_json_results.items():
        if patient_number in processed_data:
            if processed_data[patient_number] == expected_data:
                print(f"Patient {patient_number}: Results match expected output.")
            else:
                print(f"Patient {patient_number}: Results do not match expected output.")
        else:
            print(f"Patient {patient_number} not found in the processed data.")

    for patient_number in processed_data:
        if patient_number not in given_json_results:
            print(f"Patient {patient_number} in processed data but not in expected results.")

    for patient_number in given_json_results:
        if patient_number not in processed_data:
            print(f"Patient {patient_number} in expected results but not in processed data.")
else:
    print("Failed to retrieve processed or expected data.")

Patient E9559: Results match expected output.
Patient E9385: Results match expected output.
Patient E3067: Results match expected output.
Patient E9422: Results match expected output.
Patient E8661: Results match expected output.
Patient E6235: Results match expected output.
Patient E4451: Results match expected output.
Patient E8433: Results match expected output.
Patient E7593: Results match expected output.
Patient E3296: Results match expected output.
Patient E4157: Results match expected output.
Patient E7702: Results match expected output.
Patient E8158: Results match expected output.
Patient E4795: Results match expected output.
Patient E8800: Results match expected output.
Patient E4855: Results match expected output.
Patient E9691: Results match expected output.
Patient E4535: Results match expected output.
Patient E2360: Results match expected output.
Patient E3447: Results match expected output.
Patient E8208: Results match expected output.
Patient E4428: Results match expec

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---